# Stitching Tutorial

The Workflow of the Stitching Pipeline can be seen in the following. Note that the image comes from the [OpenCV Documentation](https://docs.opencv.org/3.4/d1/d46/group__stitching.html).

With the following block, we allow displaying resulting images within the notebook:

In [1]:
import matplotlib.pyplot as plt
import cv2 as cv
import numpy as np
from pathlib import Path

from stitching.image_handler import ImageHandler
from stitching.feature_detector import FeatureDetector
from stitching.feature_matcher import FeatureMatcher
from stitching.subsetter import Subsetter
from stitching.camera_estimator import CameraEstimator
from stitching.camera_adjuster import CameraAdjuster
from stitching.camera_wave_corrector import WaveCorrector
from stitching.warper import Warper
from stitching.timelapser import Timelapser
from stitching.stitching_error import StitchingError

import signal

In [2]:
def get_image_paths(img_set):
    return sorted([str(path.relative_to('.')) for path in Path('../../data/sequential_training').rglob(img_set)])

In [3]:
def get_displacement(img_paths):
    img_handler = ImageHandler()
    img_handler.set_img_names(img_paths)

    medium_imgs = list(img_handler.resize_to_medium_resolution())
    final_imgs = list(img_handler.resize_to_final_resolution())

    finder = FeatureDetector()
    features = [finder.detect_features(img) for img in medium_imgs]

    matcher = FeatureMatcher()
    matches = matcher.match_features(features)

    subsetter = Subsetter()

    indices = subsetter.get_indices_to_keep(features, matches)

    medium_imgs = subsetter.subset_list(medium_imgs, indices)
    final_imgs = subsetter.subset_list(final_imgs, indices)
    features = subsetter.subset_list(features, indices)
    matches = subsetter.subset_matches(matches, indices)

    camera_estimator = CameraEstimator()
    camera_adjuster = CameraAdjuster()
    wave_corrector = WaveCorrector()

    cameras = camera_estimator.estimate(features, matches)
    cameras = camera_adjuster.adjust(features, matches, cameras)
    cameras = wave_corrector.correct(cameras)

    warper = Warper()
    warper.set_scale(cameras)

    final_sizes = img_handler.get_final_img_sizes()
    camera_aspect = img_handler.get_medium_to_final_ratio()    # since cameras were obtained on medium imgs

    warped_final_imgs = list(warper.warp_images(final_imgs, cameras, camera_aspect))
    final_corners, final_sizes = warper.warp_rois(final_sizes, cameras, camera_aspect)

    def get_first_nonzero_pixel(ndarr):
        return np.argwhere(ndarr > 0)[0][:2]

    def get_center(corner, size):
        return (round(corner[0] + size[1] / 2), round(corner[1] + size[0] / 2))

    timelapser = Timelapser('as_is')
    timelapser.initialize(final_corners, final_sizes)
    centers = []
    for img, corner, size in zip(warped_final_imgs, final_corners, final_sizes):
        timelapser.process_frame(img, corner)
        frame = timelapser.get_frame()
        centers.append(get_center(get_first_nonzero_pixel(frame), size))
    
    centers = np.array(centers)
    return indices, np.diff(centers, axis=0)

In [4]:
sequential_imgs = get_image_paths('*.jpg')

In [5]:
get_displacement(sequential_imgs[:10])

(array([3, 4, 5, 6, 7, 8, 9], dtype=int32), array([[   0,  112],
        [ 112, -112],
        [   0,  112],
        [-112,  112],
        [   0,  112],
        [ 112, -112]]))

In [6]:
def timeout(signum, _):
    raise TimeoutError("Timeout")

In [7]:
disps = []
buckets = [[-1, -1]]

window_size = 3
signal.signal(signal.SIGALRM, timeout)

for i in range(min(len(sequential_imgs) - window_size, 10**7)):
    try:
        signal.alarm(1)
        window = sequential_imgs[i:i+window_size]
        indices, _ = get_displacement(window)
        indices = indices + i
        
        if buckets[-1][1] in indices:
            buckets[-1][1] = indices[-1]
        else:
            print(f'Contiguous stitching {buckets[-1][0]}-{buckets[-1][1]}')
            buckets.append([indices[0], indices[-1]])

        signal.alarm(0)
    except TimeoutError:
        print('timeout - stitching failed')
    except StitchingError:
        print('stitching failed')
    except Exception as e:
        if "opencv" in str(e):
            print('opencv error')
        else:
            print(e)

buckets = buckets[1:]  # drop placeholder
# disps = np.array(disps)
# disps

Contiguous stitching -1--1
Contiguous stitching 0-2
stitching failed
Contiguous stitching 3-6
stitching failed
Contiguous stitching 7-10
Contiguous stitching 12-15
Contiguous stitching 16-19
Contiguous stitching 20-23
Contiguous stitching 24-30
Contiguous stitching 31-34
Contiguous stitching 35-41
stitching failed
Contiguous stitching 42-46
Contiguous stitching 47-53
Contiguous stitching 52-57
timeout - stitching failed
Contiguous stitching 58-63
Contiguous stitching 64-67
Contiguous stitching 68-71
Contiguous stitching 72-83
Contiguous stitching 84-87
Contiguous stitching 88-91
Contiguous stitching 92-95
stitching failed
Contiguous stitching 96-109
Contiguous stitching 110-121
timeout - stitching failed
Contiguous stitching 122-125
Contiguous stitching 126-129
Contiguous stitching 130-133
Contiguous stitching 134-138
Contiguous stitching 139-140
Contiguous stitching 141-144
stitching failed
Contiguous stitching 145-148
Contiguous stitching 149-152
Contiguous stitching 153-156
Contiguo

In [9]:
sorted(buckets, key=lambda x: x[1]-x[0], reverse=True)

[[1201, 1229],
 [3314, 3338],
 [400, 421],
 [1754, 1773],
 [2632, 2651],
 [1066, 1084],
 [1389, 1407],
 [828, 845],
 [1998, 2015],
 [244, 260],
 [748, 764],
 [2755, 2771],
 [3702, 3718],
 [289, 304],
 [1443, 1458],
 [1559, 1574],
 [1588, 1603],
 [2245, 2260],
 [3106, 3121],
 [3206, 3221],
 [475, 489],
 [655, 669],
 [1261, 1275],
 [1692, 1706],
 [1977, 1991],
 [2043, 2057],
 [2414, 2428],
 [2504, 2518],
 [2899, 2913],
 [2939, 2953],
 [96, 109],
 [444, 457],
 [1008, 1021],
 [1532, 1545],
 [1628, 1641],
 [1908, 1921],
 [2834, 2847],
 [3128, 3141],
 [3155, 3168],
 [3498, 3511],
 [1426, 1438],
 [2272, 2284],
 [2304, 2316],
 [2359, 2371],
 [2467, 2479],
 [3277, 3289],
 [3374, 3386],
 [3392, 3404],
 [72, 83],
 [110, 121],
 [224, 235],
 [273, 284],
 [462, 473],
 [693, 704],
 [1125, 1136],
 [1953, 1964],
 [2149, 2160],
 [2862, 2873],
 [2914, 2925],
 [3194, 3205],
 [3295, 3306],
 [3441, 3452],
 [3660, 3671],
 [173, 183],
 [261, 271],
 [496, 506],
 [532, 542],
 [916, 926],
 [1308, 1318],
 [1714, 

In [12]:
seq_buckets = [[b1[0], b2[-1]] for b1, b2 in zip(buckets, buckets[1:]) if b1[1] == b2[0] - 1]
len(seq_buckets)

548

In [14]:
# np.save('buckets.npy', np.array(buckets))
# np.savetxt('buckets.txt', np.array(buckets), fmt='%d')